## Wasserstein

In [97]:
import wasserstein as ws

In [1282]:
quantity_map_normalized.loc[100008].iloc[quantity_map_normalized.loc[100008].nonzero()]

1526     113.333333
18752      6.300000
18868    340.000000
4343     226.400000
Name: 100008, dtype: float64

In [1268]:
recipes_pruned = np.load('recipes_pruned.npy')
recipes_pruned = [recipe for recipe in recipes_pruned.tolist()]

In [1261]:
remove = [ingredient for ingredient in ingredients if ingredient not in ingredients_pruned]

In [1515]:
qmnp = quantity_map.drop(labels=remove, axis='columns')
qmnp = qmnp.div(qmnp.sum(axis=1), axis=0)

In [ ]:
qmnp.to_csv()

In [1263]:
quantity_map_pruned = quantity_map.drop(labels=remove, axis='columns')
quantity_map_pruned.shape

(41797, 2406)

In [1269]:
quantity_map_pruned = quantity_map_pruned.loc[recipes_pruned]

In [226]:
quantity_map_normalized_pruned = quantity_map_pruned.div(quantity_map.sum(axis=1), axis=0)
quantity_map_normalized_pruned = quantity_map_normalized_pruned.loc[recipes_pruned]

In [1270]:
quantity_map_pruned.shape

(41791, 2406)

In [1526]:
from scipy.stats import wasserstein_distance

In [1528]:
recipe_id = 131102
recipe = 100008

In [1529]:
yi1 = quantity_map_normalized_pruned.loc[recipe_id].iloc[quantity_map_normalized_pruned.loc[recipe_id].nonzero()].index
yi2 = quantity_map_normalized_pruned.loc[recipe].iloc[quantity_map_normalized_pruned.loc[recipe].nonzero()].index
        

In [1531]:
y = X[[corpus.index(i1) for i1 in yi1]]
z = X[[corpus.index(i1) for i1 in yi2]]

In [1634]:
d1 = quantity_map_normalized_pruned.loc[recipe_id].iloc[quantity_map_normalized_pruned.loc[recipe_id].nonzero()]
d2 = quantity_map_normalized_pruned.loc[recipe].iloc[quantity_map_normalized_pruned.loc[recipe].nonzero()]

In [ ]:
dict_wass = {}
for recipe in tqdm(quantity_map_normalized_pruned.index):
    yi1 = quantity_map_normalized_pruned.loc[recipe_id].iloc[quantity_map_normalized_pruned.loc[recipe_id].nonzero()].index
    yi2 = quantity_map_normalized_pruned.loc[recipe].iloc[quantity_map_normalized_pruned.loc[recipe].nonzero()].index
    y = X[[corpus.index(i1) for i1 in yi1]]
    z = X[[corpus.index(i1) for i1 in yi2]]
    d1 = quantity_map_normalized_pruned.loc[recipe_id].iloc[quantity_map_normalized_pruned.loc[recipe_id].nonzero()]
    d2 = quantity_map_normalized_pruned.loc[recipe].iloc[quantity_map_normalized_pruned.loc[recipe_id].nonzero()]
    wass_dist = wasserstein_distance(y, z, d1, d2)
    dict_wass.update({recipe:wass_dist})

In [2632]:
np.array([int(a) for a in list(d1.index)])

array([ 1526, 16278, 16317, 16401, 16424])

In [1666]:
histograms = quantity_map_normalized_pruned.apply(lambda x: x.iloc[x.nonzero()].to_dict(), axis=1)#x.iloc[x.nonzero()], axis=1)

In [1700]:
quantity_map_normalized_pruned.to_csv('qmnp.csv')

In [1894]:
from itertools import product
from collections import defaultdict

import numpy as np
from scipy.spatial.distance import euclidean
import pulp


singleindexing = lambda m, i, j: m*i+j
unpackindexing = lambda m, k: (k/m, k % m)


def tokens_to_fracdict(tokens):
    cntdict = defaultdict(lambda : 0)
    for token in tokens:
        cntdict[token] += 1
    totalcnt = sum(cntdict.values())
    return {token: float(cnt)/totalcnt for token, cnt in cntdict.items()}


# use PuLP
def word_mover_distance_probspec(recipe_1, recipe_2, wvmodel, histograms, lpFile=None):
    #d1 = quantity_map_normalized_pruned.loc[recipe1].iloc[quantity_map_normalized_pruned.loc[recipe1].nonzero()]
    #d2 = quantity_map_normalized_pruned.loc[recipe2].iloc[quantity_map_normalized_pruned.loc[recipe2].nonzero()]
    
    all_tokens = list(set(list(histograms[recipe_1].keys())+list(histograms[recipe_2].keys())))
    wordvecs = {token: wvmodel[token] for token in all_tokens}
    
    first_sent_buckets = histograms[recipe_1]#tokens_to_fracdict(first_sent_tokens)
    second_sent_buckets = histograms[recipe_2]#tokens_to_fracdict(second_sent_tokens)

    T = pulp.LpVariable.dicts('T_matrix', list(product(all_tokens, all_tokens)), lowBound=0)
    prob = pulp.LpProblem('WMD', sense=pulp.LpMinimize)
    prob += pulp.lpSum([T[token1, token2]*euclidean(wordvecs[token1], wordvecs[token2])
                        for token1, token2 in product(all_tokens,all_tokens)])
    for token2 in second_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token1 in first_sent_buckets])==second_sent_buckets[token2]
    for token1 in first_sent_buckets:
        prob += pulp.lpSum([T[token1, token2] for token2 in second_sent_buckets])==first_sent_buckets[token1]

    if lpFile!=None:
        prob.writeLP(lpFile)

    prob.solve()
    return prob


def word_mover_distance(first_sent_tokens, second_sent_tokens, wvmodel, histograms, lpFile=None):
    prob = word_mover_distance_probspec(first_sent_tokens, second_sent_tokens, wvmodel, histograms, lpFile=lpFile)
    return pulp.value(prob.objective)

In [1694]:
recipe_ids = list(quantity_map_normalized_pruned.index)

In [1695]:
word_mover_distance(100008, 10011, model)

{'1526': 0.16520091346387442, '18752': 0.009183227248433023, '18868': 0.4956027403916234, '4343': 0.3300131188960692} {'1526': 0.2128836340488589, '16157': 0.2128836340488589, '16317': 0.0006261283354378203, '16421': 0.0006574347522097113, '16424': 0.005259478017677691, '1684': 0.567689690796957}


7.0087552102746615

In [1696]:
np.save('wass_dict', dict_wass)

In [ ]:
recipe_id = 100008
dict_wass={}
for recipe in tqdm(recipe_ids):
    dict_wass.update({recipe:word_mover_distance(recipe_id, recipe, model)})

In [2602]:
dict_wass = np.load('wass_distances.npy').item()

In [2603]:
import operator
sorted_x = sorted(dict_wass.items(), key=operator.itemgetter(1))

In [2613]:
word_mover_distance(79735,218999, model, histograms)

0.39994472670360354

In [2604]:
sorted_x[:10]

[(100008, 0.0),
 (192623, 1.8203961853853965),
 (14050, 1.8586213241406908),
 (79735, 2.402067929167796),
 (218999, 2.45137191770702),
 (222931, 2.6085593810067516),
 (256240, 2.6128663702327115),
 (238477, 2.6189217097432516),
 (215117, 2.646089191237682),
 (258377, 2.67058558927953)]

In [2617]:
for recipe in lista:
    pprint(recipes.loc[recipe]['ingredients'])

[{'id': '18868', 'text': '1/2 cup rice vinegar'},
 {'id': '1526', 'text': '1/2 cup white sugar'},
 {'id': '4342', 'text': '1 clove garlic, minced'},
 {'id': '16421', 'text': '1/4 teaspoon salt'},
 {'id': '20244', 'text': '1 1/2 teaspoons dried red pepper flakes'}]
[{'id': '18868', 'text': '1/2 cup rice wine vinegar'},
 {'id': '1526', 'text': '1/2 cup white sugar'},
 {'id': '16421', 'text': '2 teaspoons salt'},
 {'id': '26958',
  'text': '1 tablespoon chili garlic sauce (such as Lee Kum Kee®)'}]


In [249]:
quantity_map_normalized_pruned.loc[131102].iloc[quantity_map_normalized_pruned.loc[131102].nonzero()]

1526     0.887951
16278    0.000010
16317    0.062679
16401    0.005484
16424    0.043875
Name: 131102, dtype: float64

In [237]:
recipes.loc[131102]['ingredients']

[{'id': '16317', 'text': '12 eggs'},
 {'id': '1526', 'text': '1/2 cup white sugar'},
 {'id': '16278', 'text': '2 quarts milk'},
 {'id': '16424', 'text': '2 teaspoons vanilla extract'},
 {'id': '16401', 'text': '1/4 teaspoon ground nutmeg'}]

In [238]:
recipes.loc[259907]['ingredients']

[{'id': '1526', 'text': '1 cup white sugar'},
 {'id': '34567', 'text': '1 cup prickly pear juice'}]

In [239]:
recipes.loc[234946]['ingredients']

[{'id': '21757', 'text': '1 quart lime sherbet'},
 {'id': '8855',
  'text': '1 (2 liter) bottle diet lemon-lime carbonated beverage'}]

In [244]:
recipes.loc[153344]['ingredients']

[{'text': '1 quart cold milk', 'id': '16278'},
 {'text': '2 pkg. (4 serving size) JELL-O Vanilla Flavor Instant Pudding & Pie Filling',
  'id': '21356'},
 {'text': '32 NILLA Wafers, finely crushed, divided', 'id': '15993'},
 {'text': '8 paper or plastic cups (6 to 7 oz.)', 'id': '24575'},
 {'text': '8 worm-shaped chewy fruit snacks', 'id': '20672'}]

In [ ]:
sorted_x

In [344]:
recipes.loc[recipes.index[2]]['ingredients']

[{'text': '2 tablespoons vegetable oil', 'id': '6305'},
 {'text': '1 teaspoon minced garlic', 'id': '4342'},
 {'text': '1 large onion, chopped', 'id': '4397'},
 {'text': '1 (12 ounce) pork tenderloin, cut into 1/2 inch cubes',
  'id': '4175'},
 {'text': '1 (19 ounce) can black beans, drained and rinsed', 'id': '2779'},
 {'text': '1/4 cup water', 'id': '2496'},
 {'text': '1 1/2 cups chicken stock', 'id': '13968'},
 {'text': '3 chorizo sausages, cut into 1/2 inch thick pieces', 'id': '5821'},
 {'text': '2 bay leaves', 'id': '16380'},
 {'text': 'salt and pepper to taste', 'id': '16421'}]